In [ ]:
#base file path
path = '/content/drive/Shareddrives/soft computing project test'

# Imports and helper functions

In [ ]:
import os                                 # For file and directory operations
import cv2                                # For image processing (reading and resizing images)
import numpy as np                        # For numerical operations (e.g., reshaping arrays)
import pandas as pd                       # For data manipulation (reading CSV, manipulating data)
from sklearn.model_selection import train_test_split  # For splitting data into training and testing sets
from sklearn.utils import shuffle          # For shuffling data
from tensorflow.keras import layers, models  # For creating and managing deep learning models
from tensorflow.keras.models import Model  # For managing models
import tensorflow as tf                   # For deep learning operations
from imblearn.over_sampling import SMOTE  # For oversampling imbalanced datasets
from google.colab import drive            # For mounting Google Drive


# Mount Google Drive
drive.mount('/content/drive')
driveLocation = path
%cd $driveLocation

In [ ]:
#function to load images from a directory
def load_images_from_folder(folder_path):
    dataset = np.load(folder_path)
    return dataset['images'], dataset['image_files']

In [ ]:
#function to load the clusters
def get_clusters(image_files, input_file='cluster_assignments.csv'):
    df = pd.read_csv(input_file)
    # map the clusters back to the provided image files
    cluster_dict = dict(zip(df['Image File'], df['Cluster']))

    # ensure clusters are returned in the same order as image_files
    clusters = [cluster_dict[img] for img in image_files]

    return clusters

# load the dataset

In [ ]:
#load images from the unlabeled dataset
unlabeled_dataset_folder = os.path.join(path, 'dataset.npz')
images, image_files = load_images_from_folder(unlabeled_dataset_folder)
# Let's assume cluster 0 corresponds to 'no ship' and cluster 1 corresponds to 'ship'

#get the clusters
clusters = get_clusters(image_files, 'cluster_assignments_kmeans.csv')

# Convert labels to a binary format (0 or 1)
labels = np.array(clusters)  # Cluster 0 as 'no ship', Cluster 1 as 'ship'


#count how many images are in each class
unique_labels, counts = np.unique(labels, return_counts=True)

# Print the results
for label, count in zip(unique_labels, counts):
    print(f"Label {label}: {count} images")


# Split the data into training and a temporary set for validation and testing
X_train, X_temp, y_train, y_temp = train_test_split(images, labels, test_size=0.3, random_state=42)

# Create SMOTE object
smote = SMOTE(random_state=42)

# Reshape the image data if necessary
X_train_flattened = X_train.reshape(X_train.shape[0], -1)  # Reshape to (num_samples, 80*80*3)

# Apply SMOTE to oversample the minority class
X_resampled, y_resampled = smote.fit_resample(X_train_flattened, y_train)

#Reshape the oversampled images back to their original dimensions
X_train_balanced = X_resampled.reshape(X_resampled.shape[0], 80, 80, 3)  # Reshape back to (num_samples, 80, 80, 3)

# Shuffle the balanced training set
X_train_balanced, y_train_balanced = shuffle(X_train_balanced, y_resampled, random_state=42)

#  Now split the temporary set into validation and test sets (70% val, 30% test of the temp set)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.3, random_state=42)

print(f"Number of images in the training set: {len(X_train_balanced)}")
print(f"Number of images in the validation set: {len(X_val)}")
print(f"Number of images in the testing set: {len(X_test)}")

# Create TensorFlow Dataset objects from the NumPy arrays
train_dataset = tf.data.Dataset.from_tensor_slices((X_train_balanced, y_train_balanced))
val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val))
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))

# Normalize the images (scaling between 0 and 1)
def normalize(image, label):
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

# Apply normalization lazily on each batch
train_dataset = train_dataset.map(normalize)
val_dataset = val_dataset.map(normalize)
test_dataset = test_dataset.map(normalize)

# Set the batch size and shuffle the training dataset
batch_size = 32
train_dataset = train_dataset.batch(batch_size).shuffle(buffer_size=1000)
val_dataset = val_dataset.batch(batch_size)
test_dataset = test_dataset.batch(batch_size)

# Evolve the genetic algorithm to find the optimal model architecture

In [ ]:
# Genetic Algorithm Configuration
POPULATION_SIZE = 10
GENERATIONS = 5
MUTATION_RATE = 0.1

# Define possible configurations
activation_type_conv_layers = ['relu', 'sigmoid', 'tanh']
activation_type_dense_layers = ['relu', 'sigmoid', 'tanh']
pooling_types = [None, 'max', 'average'] # pooling type for each convolution block
num_conv_layers_block1 = range(1, 4)  # 1 to 3 convolution layers per block
batch_norm_block1 = [True, False]
num_conv_layers_block2 = range(1, 4)
batch_norm_block2 = [True, False]
num_conv_layers_block3 = range(1, 4)
batch_norm_block3 = [True, False]
num_dense_layers_options = range(1, 4)  # 1 to 3 dense layers
num_of_nodes = [16, 32, 64]  # Nodes per dense layer
dropout_options = [0.0, 0.3, 0.5]  # Dropout rate options

class Individual:
    def __init__(self):
        # Randomly choose configuration per block
        self.num_conv_layers_block1 = np.random.choice(num_conv_layers_block1)
        self.batch_norm_block1 = np.random.choice(batch_norm_block1)
        self.num_conv_layers_block2 = np.random.choice(num_conv_layers_block2)
        self.batch_norm_block2 = np.random.choice(batch_norm_block2)
        self.num_conv_layers_block3 = np.random.choice(num_conv_layers_block3)
        self.batch_norm_block3 = np.random.choice(batch_norm_block3)

        # Randomly choose activation for conv and dense layers
        self.conv_activation = np.random.choice(activation_type_conv_layers)
        self.dense_activation = np.random.choice(activation_type_dense_layers)

        # Pooling type for each convolutional block
        self.pooling_layers = [np.random.choice(pooling_types) for _ in range(3)]

        # Dense layer configurations
        self.num_dense_layers = np.random.choice(num_dense_layers_options)
        self.num_nodes_per_layer = np.random.choice(num_of_nodes, self.num_dense_layers)

        # Dropout
        self.dropout_rate = np.random.choice(dropout_options)

        # Model and fitness attributes
        self.model = None
        self.fitness = 0

    def add_conv_block(self, model, num_layers, use_batch_norm, pooling_type):
        for _ in range(num_layers):
            model.add(layers.Conv2D(32, (3, 3), activation=self.conv_activation, padding='same'))
            if use_batch_norm:
                model.add(layers.BatchNormalization())
        if pooling_type == 'max':
            model.add(layers.MaxPooling2D((2, 2)))
        elif pooling_type == 'average':
            model.add(layers.AveragePooling2D((2, 2)))

    def build_model(self):
        model = models.Sequential()
        model.add(layers.InputLayer(input_shape=(80, 80, 3)))

        # Add convolution blocks with respective configurations
        self.add_conv_block(model, self.num_conv_layers_block1, self.batch_norm_block1, self.pooling_layers[0])
        self.add_conv_block(model, self.num_conv_layers_block2, self.batch_norm_block2, self.pooling_layers[1])
        self.add_conv_block(model, self.num_conv_layers_block3, self.batch_norm_block3, self.pooling_layers[2])

        model.add(layers.Flatten())

        # Add dense layers with respective configurations
        for nodes in self.num_nodes_per_layer:
            model.add(layers.Dense(nodes, activation=self.dense_activation))
            if self.dropout_rate > 0:
                model.add(layers.Dropout(self.dropout_rate))

        model.add(layers.Dense(2, activation='softmax'))  # Output layer for binary classification
        self.model = model
        return model

    def evaluate(self):
        self.model = self.build_model()
        self.model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
        self.model.fit(train_dataset, epochs=10, validation_data=val_dataset, verbose=0)
        self.fitness = self.model.evaluate(test_dataset, verbose=0)[1]  # Get accuracy
        print(f"Evaluated Individual - Fitness: {self.fitness:.4f}")

def mutate(individual):
    # Randomly mutate an individual's properties for convolutional blocks
    if np.random.rand() < MUTATION_RATE:
        individual.num_conv_layers_block1 = np.random.choice(num_conv_layers_block1)
    if np.random.rand() < MUTATION_RATE:
        individual.num_conv_layers_block2 = np.random.choice(num_conv_layers_block2)
    if np.random.rand() < MUTATION_RATE:
        individual.num_conv_layers_block3 = np.random.choice(num_conv_layers_block3)

    # Mutate batch normalization settings
    if np.random.rand() < MUTATION_RATE:
        individual.batch_norm_block1 = np.random.choice(batch_norm_block1)
    if np.random.rand() < MUTATION_RATE:
        individual.batch_norm_block2 = np.random.choice(batch_norm_block2)
    if np.random.rand() < MUTATION_RATE:
        individual.batch_norm_block3 = np.random.choice(batch_norm_block3)

    # Mutate activation functions
    if np.random.rand() < MUTATION_RATE:
        individual.conv_activation = np.random.choice(activation_type_conv_layers)
    if np.random.rand() < MUTATION_RATE:
        individual.dense_activation = np.random.choice(activation_type_dense_layers)

    # Mutate pooling types for each block
    if np.random.rand() < MUTATION_RATE:
        individual.pooling_layers = [np.random.choice(pooling_types) for _ in range(3)]

    # Mutate dense layer properties
    if np.random.rand() < MUTATION_RATE:
        individual.num_dense_layers = np.random.choice(num_dense_layers_options)
    if np.random.rand() < MUTATION_RATE:
        individual.num_nodes_per_layer = np.random.choice(num_of_nodes, individual.num_dense_layers)

    # Mutate dropout rate
    if np.random.rand() < MUTATION_RATE:
        individual.dropout_rate = np.random.choice(dropout_options)

def crossover(parent1, parent2):
    # Create a new individual by combining the properties of two parents
    child = Individual()

    # Crossover for convolutional block layers
    child.num_conv_layers_block1 = np.random.choice([parent1.num_conv_layers_block1, parent2.num_conv_layers_block1])
    child.num_conv_layers_block2 = np.random.choice([parent1.num_conv_layers_block2, parent2.num_conv_layers_block2])
    child.num_conv_layers_block3 = np.random.choice([parent1.num_conv_layers_block3, parent2.num_conv_layers_block3])

    # Crossover for batch normalization settings
    child.batch_norm_block1 = np.random.choice([parent1.batch_norm_block1, parent2.batch_norm_block1])
    child.batch_norm_block2 = np.random.choice([parent1.batch_norm_block2, parent2.batch_norm_block2])
    child.batch_norm_block3 = np.random.choice([parent1.batch_norm_block3, parent2.batch_norm_block3])

    # Crossover for activation functions
    child.conv_activation = parent1.conv_activation if np.random.rand() > 0.5 else parent2.conv_activation
    child.dense_activation = parent1.dense_activation if np.random.rand() > 0.5 else parent2.dense_activation

    # Crossover for pooling layers and dense layers
    child.pooling_layers = parent1.pooling_layers if np.random.rand() > 0.5 else parent2.pooling_layers
    child.num_dense_layers = np.random.choice([parent1.num_dense_layers, parent2.num_dense_layers])
    child.num_nodes_per_layer = parent1.num_nodes_per_layer if np.random.rand() > 0.5 else parent2.num_nodes_per_layer

    # Crossover for dropout rate
    child.dropout_rate = parent1.dropout_rate if np.random.rand() > 0.5 else parent2.dropout_rate

    return child

# Initialize the population
population = [Individual() for _ in range(POPULATION_SIZE)]

# Genetic Algorithm Process
for generation in range(GENERATIONS):
    # Evaluate fitness of each individual
    for individual in population:
        individual.evaluate()

    # Sort population by fitness
    population.sort(key=lambda ind: ind.fitness, reverse=True)

    # Print best architecture and its fitness
    best_individual = population[0]
    print(f"Generation {generation}, Best fitness: {best_individual.fitness:.4f}, Architecture: {vars(best_individual)}")

    # Create new population
    new_population = population[:2]  # Keep the top 2
    while len(new_population) < POPULATION_SIZE:
        parent1, parent2 = np.random.choice(population[:5], 2)  # Select parents from top individuals
        child = crossover(parent1, parent2)
        mutate(child)
        new_population.append(child)

    population = new_population  # Move to the next generation

print("Optimization Complete")
